In [2]:
# execute this cell before you start

import tensorflow as tf
from tensorflow.keras import layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

print(tf.VERSION)
print(tf.keras.__version__)


1.12.0
2.1.6-tf


#  CA2
## due on 8/03/2019

to submit the assignment, please do the following:

- do `Cell -> All output -> Clear` to clear all your output
- save the notebook (CA3.ipynb)

# The Reuters newswire data

Consider the data in  `keras.datasets.reuters` and train a network which reliably categorizes the newswires. 

Load the data, restricting the number of words to the most frequent 10,000 words

In [3]:
num_words=1000
reuters = keras.datasets.reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=num_words)


`num_classes` is the number of unique labels in the data

In [4]:
num_classes = len(set(train_labels))

Inspect the data

In [8]:
len(train_data), len(test_labels)

(8982, 2246)

Get the word index

In [39]:
word_index = reuters.get_word_index()

Index 0 is unused. Use it to set the reserved `<PAD>` used later on.

In [40]:
word_index["<PAD>"] = 0

The word index maps words to the word code. So create a reverse map from word code to word and a `decode_article` function so that it allows us to view articles in human readable form.

In [41]:
reverse_word_index = {}
for key, value in word_index.items():
    reverse_word_index[value] = key

def decode_article(article):
    decodedArticle = ""
    for word_code in article:
        decodedArticle += " "
        decodedArticle += reverse_word_index.get(word_code, "?")
    return decodedArticle

To have uniform input data, we make sure every article is a uniform 256 word long and pad the data with `<PAD>` when necessary

In [44]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], maxlen=256, padding="post")
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], maxlen=256, padding="post")

decode_article(train_data[0])

' the of of mln loss for plc said at only ended said of could 1 traders now april 0 a after said from 1985 and from foreign 000 april 0 prices its account year a but in this mln home an states earlier and rise and revs vs 000 its 16 vs 000 a but 3 of of several and shareholders and dividend vs 000 its all 4 vs 000 1 mln agreed of april 0 are 2 states will billion total and against 000 pct dlrs <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PA

Now build the model, compile it and train it

In [45]:
vocab_size = num_words
model = keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(512, activation=tf.nn.relu))
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x=train_data, y=train_labels, epochs=40)
test_loss, test_acc = model.evaluate(x=test_data, y=test_labels)
test_loss, test_acc

/home/neeraj/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/40
8982/8982 [==============================] - 2s 168us/step - loss: 2.4231 - acc: 0.3634
Epoch 2/40
8982/8982 [==============================] - 0s 53us/step - loss: 1.8901 - acc: 0.4876
Epoch 3/40
8982/8982 [==============================] - 0s 53us/step - loss: 1.6622 - acc: 0.5819
Epoch 4/40
8982/8982 [==============================] - 1s 62us/step - loss: 1.5390 - acc: 0.6146
Epoch 5/40
8982/8982 [==============================] - 1s 66us/step - loss: 1.4464 - acc: 0.6378
Epoch 6/40
8982/8982 [==============================] - 1s 62us/step - loss: 1.3472 - acc: 0.6659
Epoch 7/40
8982/8982 [==============================] - 1s 64us/step - loss: 1.2461 - acc: 0.6903
Epoch 8/40
8982/8982 [==============================] - 1s 63us/step - loss: 1.1596 - acc: 0.7129
Epoch 9/40
8982/8982 [==============================] - 1s 64us/step - loss: 1.0896 - acc: 0.7298
Epoch 10/40
8982/8982 [==============================] - 1s 63us/step - loss: 1.0373 - acc: 0.7419
Epoch 11/40
8982/8

(1.1568343996046275, 0.74621549421193234)